In [1]:
%matplotlib inline

import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'src/')))
from sgpi.util import get_dataset, get_xy, get_r_squared, split, to_data_matrix

from sgpi import model
from sgpi.learner import SGRegressionLearner
import sqlalchemy as sa
from sqlalchemy import orm, func, or_, and_

from sgpi.plot import *

import pandas as pd
import pysgpp as sg; sg.omp_set_num_threads(4)

In [8]:
def results_for_level():
    ses = model.make_session()
    results = ses.query(model.Result, func.min(model.Result.validation_mse)).\
        join(model.RegularizationConfig).\
        join(model.Experiment).\
        join(model.GridConfig).\
        filter(model.Result.experiment_id.between(40, 200)).\
        filter(or_(model.RegularizationConfig.type == sg.RegularizationType_ElasticNet,
                  model.RegularizationConfig.type == sg.RegularizationType_GroupLasso,
                  model.RegularizationConfig.type == sg.RegularizationType_Lasso)).\
        filter(model.RegularizationConfig.l1_ratio != None).\
        filter(model.Experiment.dataset == 'concrete').\
        filter(model.GridConfig.level.between(4,5)).\
        group_by(model.Result.experiment_id).\
        order_by(model.GridConfig.level, model.Result.validation_mse)
    return results

# Results missing:
None

In [9]:
results_lasso = results_for_level()

In [10]:
def type_to_str(type, l1_ratio):
    if type == sg.RegularizationType_ElasticNet:
        if l1_ratio == 1.0:
            return "Lasso"
        return "Elastic Net ($\\lambda_2$ = {})".format(l1_ratio)
    types = {sg.RegularizationType_GroupLasso: 'Group Lasso'}
    return types[type]

#for res, mse in results_lasso:
#    print "{}, {}, {:2.3e}, {:2.6f}".format(res.grid_config.level, type_to_str(res.regularization_config.type, res.regularization_config.l1_ratio),\
#                                       res.regularization_config.lambda_reg, mse)
    
for res, mse in results_lasso:
    reg_type = type_to_str(res.regularization_config.type, res.regularization_config.l1_ratio)
    print "{} & {} & {:1.0e} & {:2.3f} & {} & {:2.3f} & {:2.3f}\\\\".\
        format(res.grid_config.level, reg_type, res.regularization_config.lambda_reg,
                np.sqrt(res.validation_mse), int(res.train_grid_points), np.sqrt(res.train_mse), np.sqrt(res.test_mse))

4 & Group Lasso & 1e-02 & 4.774 & 1385 & 3.133 & 4.032\\
4 & Elastic Net ($\lambda_2$ = 0.95) & 1e-02 & 4.868 & 1382 & 2.902 & 3.869\\
4 & Lasso & 1e-02 & 4.884 & 1382 & 2.911 & 3.850\\
4 & Elastic Net ($\lambda_2$ = 0.5) & 1e-02 & 4.927 & 1384 & 3.077 & 4.046\\
4 & Elastic Net ($\lambda_2$ = 0.05) & 1e-03 & 5.060 & 1367 & 2.483 & 3.813\\
5 & Lasso & 1e-02 & 4.582 & 6632 & 2.470 & 3.737\\
5 & Elastic Net ($\lambda_2$ = 0.95) & 1e-02 & 4.594 & 6632 & 2.460 & 3.742\\
5 & Lasso & 5e+00 & 4.637 & 6401 & 2.216 & 3.759\\
5 & Group Lasso & 1e-02 & 4.650 & 6694 & 2.861 & 3.955\\
5 & Elastic Net ($\lambda_2$ = 0.5) & 1e-02 & 4.669 & 6633 & 2.400 & 3.844\\
5 & Elastic Net ($\lambda_2$ = 0.05) & 1e-02 & 4.730 & 6648 & 2.297 & 4.085\\
5 & Elastic Net ($\lambda_2$ = 0.95) & 1e-02 & 5.024 & 1461 & 3.330 & 4.523\\
5 & Elastic Net ($\lambda_2$ = 0.5) & 1e-02 & 5.039 & 1461 & 3.498 & 4.460\\
5 & Elastic Net ($\lambda_2$ = 0.05) & 1e-03 & 5.045 & 1461 & 2.736 & 4.218\\
5 & Elastic Net ($\lambda_2$ = 0.9

In [115]:
pd.DataFrame(np.vstack((x,y)).T)

,0,1
0,1.0,99.278064


In [28]:
for res, mse in results_lasso:
    print res.experiment_id

75
40
76
74


In [29]:
ses = model.make_session()

In [41]:
exp75 = ses.query(model.Result).filter(model.Result.experiment_id == 76).all()

In [44]:
for res in exp75:
    print res.regularization_config.lambda_reg

0.01
0.01
0.001
0.001
0.0001
0.0001
0.1
0.1
1.0
1.0


In [54]:
t = exp75[0].adaptivity_config
print t.no_points, t.num_refinements

3 5


In [61]:
print "{:f}".format(10**(-4))

0.000100
